In [3]:
import os
import base64
import requests
from requests import post, get
import json
from tqdm import tqdm
import pandas as pd
import time
import math
import re

In [102]:
client_credentials = [
    ('e0c69679b0b7452ba1e946c37ba6f764', 'efc8135d62554a0790c4b9dca7156bbc'),
    ('cdf75282fa8144dda8018c861f41a4c9', 'e7ef1fb2c0ab4d58a196e4f0b0f4fe70')
    # ('31922bf702714f66b4294cbc16d17fb6', '0e321c8a15324df8a249b7cf2b52ec76'),
    # ('ab611466a20546d0b0e2e0b8bdfb87a3', '4fdcf84f415a4d37a718da6835e23082'),
    # ('b3d5770c1b824d3ebdc805290e9abc1f', '01609af484f94eab9b77f6a84c4ecbde'),
    # ('4225b73cc1074094aaa695b5c9d5f610', '1ddb33f2f3d047ccae7e3bcc7df30c79'),
    # ('206782e057b7465b899a5b4ae6feabff', '35b491b06101488aaf1c5735638e8307'),
    # ('220266650ffb4ffc96c8cd3989c87f48', '5cc43b25e0454284b414362000edcd68'),
    # ('03c5419a6a324213850690c1820d07cd', 'fb36dc81965b4ae39e0e81215cf12124'),
    # ('afd9c9421cb84f17b45bfe0bc89c0cd1', '2d5ebf61ebe94ef2ab2f4e1cce02def8'),
    # ('da2f148a6d3c4eec9c106106f1015a0b', 'e3da3906110e4b22963423c4f018af60'),
    # ('7ce1ad74b969497d9a5c57489670d08f', 'd9907b42abbe4923bee2f7cfccf82946'),
    # ('ed335709032445beb21a6abfe00b805e', '6a1c1310bbf3416aa1acf9cb1fc33ac3'),
    # ('84be5714a8eb43289371bfea95054e2f', 'a4520f8a7ea24e47a9ccf5e911205b9f'),
    # ('326b44ed8b20460590ac84d7422099f6', '33d0b014852343ab9ffcaaafcedd8347'),
    # ('469e49ddbdb24c69bee3ab1be9b5eac1', 'dbb33a3d14194fb1af0064f5e19fe365'),
    # ('1475e2c15aa048d68a48805e5bb0a70d', '906bb9f9e18c425f9f7d4b539f080b98'),
    # ('6b61506f190e432ea639de97b81c2e23', '53be096902a746bf9cba27ea56f8059c'),
    # ('f27bf844a9cb4d208ea119e0481955b1', 'd4b377287b31482dbe1f64062f9d1854'),
    # ('74bb3ea8357a42c0a1b61af3f2ce1f81', '270ca3c440954d908fcd215ce2081b55'),
    # ('a3be98d4ae3245d382adbb93b0e08828', '68c3a02702774e779356e9b843d7c0d8')
]



credential_index = 0

def get_token(client_id, client_secret):
    auth_string = client_id + ":" + client_secret
    auth_bytes = auth_string.encode("utf-8")
    auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")

    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization": "Basic " + auth_base64,
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {"grant_type": "client_credentials"}

    response = requests.post(url, headers=headers, data=data)
    json_result = json.loads(response.content)

    if "access_token" in json_result:
        token = json_result["access_token"]
        return token
    else:
        print("Error: Failed to get access token")
        print(json_result)
        return None

def get_auth_header(token):
    return {"Authorization": "Bearer " + token}

def get_albums_for_artist(token, artist_id):
    url = f"https://api.spotify.com/v1/artists/{artist_id}/albums"
    headers = get_auth_header(token)
    params = {"include_groups": "album"}
    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        albums = json.loads(response.text)["items"]
        album_dict = {}
        for album in albums:
            album_id = album["id"]
            album_name = album["name"]
            album_tracks = album["total_tracks"]
            album_type = album["album_type"]
            album_release_date = album["release_date"]
            album_artists = [artist["name"] for artist in album["artists"]]
            album_images = album["images"]

            if album_name in album_dict:
                existing_tracks = album_dict[album_name]["total_tracks"]
                if album_tracks > existing_tracks:
                    album_dict[album_name] = {
                        "album_id": album_id,
                        "album_name": album_name,
                        "total_tracks": album_tracks,
                        "album_type": album_type,
                        "release_date": album_release_date,
                        "artists": album_artists,
                        "images": album_images
                    }
            else:
                album_dict[album_name] = {
                    "album_id": album_id,
                    "album_name": album_name,
                    "total_tracks": album_tracks,
                    "album_type": album_type,
                    "release_date": album_release_date,
                    "artists": album_artists,
                    "images": album_images
                }
        return album_dict
    else:
        print("Error: Failed to get albums for artist")
        print(response.content)
        return None
    
def get_tracks_for_album(token, album_id):
    url = f"https://api.spotify.com/v1/albums/{album_id}/tracks"
    headers = get_auth_header(token)
    params = {"market": "US"}
    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        tracks = json.loads(response.text)["items"]
        track_list = []
        for track in tracks:
            track_name = track["name"]
            track_id = track["id"]
            track_number = track["track_number"]
            track_explicit = track["explicit"]
            track_artists = [artist["name"] for artist in track["artists"]]
            track_dict = {
                "track_name": track_name,
                "track_id": track_id,
                "track_number": track_number,
                "track_explicit": track_explicit,
                "track_artists": track_artists,
            }
            track_list.append(track_dict)
        return track_list
    else:
        print("Error: Failed to get tracks for album")
        print(response.content)
        return None

# def get_audio_features(token, track_ids):
#     headers = {"Authorization": f"Bearer {token}"}
#     url = "https://api.spotify.com/v1/audio-features"
    
#     id_chunks = [track_ids[i:i+100] for i in range(0, len(track_ids), 100)]
    
#     dfs = []
#     for chunk in id_chunks:
#         params = {"ids": ",".join(chunk)}
#         response = requests.get(url, headers=headers, params=params)
#         data = response.json()["audio_features"]
#         features = pd.DataFrame(data)
#         features = features.drop(["type", "uri", "track_href", "analysis_url"], axis=1)
#         dfs.append(features)
    
#     features = pd.concat(dfs, ignore_index=True)
#     return features

def get_audio_features(token, track_ids):
    audio_features = []

    for i in range(0, len(track_ids), 50):
        chunk = track_ids[i:i + 50]
        ids = ','.join(chunk)
        url = f"https://api.spotify.com/v1/audio-features?ids={ids}"
        headers = {"Authorization": f"Bearer {token}"}
        r = requests.get(url, headers=headers)
        if r.status_code != 200:
            print(f"Error getting audio features for tracks: {r.status_code}")
            return None

        try:
            features = r.json()["audio_features"]
        except KeyError:
            features = []

        audio_features.extend(features)

    audio_features_filtered = []
    for feature in audio_features:
        if feature is not None:
            audio_features_filtered.append(feature)

    return pd.DataFrame(audio_features_filtered)



def load_checkpoint():
    if os.path.exists('checkpoint_0.json'):
        with open('checkpoint_0.json', 'r') as f:
            return json.load(f)
    else:
        return {"artist_id": "", "progress": 0}

def save_checkpoint(artist_id, progress):
    with open('checkpoint.json', 'w') as f:
        json.dump({"artist_id": artist_id, "progress": progress}, f)

def load_failed_requests():
    if os.path.exists('failed_requests.parquet_0'):
        return pd.read_parquet('failed_requests_0.parquet')
    else:
        return pd.DataFrame(columns=['Spotify ID'])

def save_failed_requests(df):
    df.to_parquet('failed_requests_0.parquet')

def refresh_token(token_start_time):
    global credential_index
    current_time = time.time()
    elapsed_time = current_time - token_start_time
    if elapsed_time >= 1800:
        credential_index = (credential_index + 1) % len(client_credentials)
        client_id, client_secret = client_credentials[credential_index]
        return get_token(client_id, client_secret), current_time
    return None, token_start_time

def sanitize_filename(filename):
    return re.sub(r'[\\/:"*?<>|]+', '_', filename)



In [4]:
with open('artist_ids_sub_dict_0.json', 'r') as f:
    artist_ids = json.load(f)
len(artist_ids)

1018

In [105]:
token = get_token('7727b6a6602f4dcf99d337ee11f0983e', '6b4958253cb544aaae1b0bf4858726ee')
token_start_time = time.time()

checkpoint = load_checkpoint()
failed_requests = load_failed_requests()

start_from = checkpoint["artist_id"] if checkpoint["artist_id"] else list(artist_ids.keys())[0]
start_index = list(artist_ids.keys()).index(start_from)


iteration_counter = 0
sleep_interval = 50

for artist_id, artist_name in list(artist_ids.items())[start_index:]:
    try:
        refreshed_token, new_token_start_time = refresh_token(token_start_time)
        if refreshed_token:
            token = refreshed_token
            token_start_time = new_token_start_time

        albums = get_albums_for_artist(token, artist_id)

        if albums is None:
            failed_requests = failed_requests.append({"Spotify ID": artist_id}, ignore_index=True)
            save_failed_requests(failed_requests)
            continue

        album_df = pd.DataFrame.from_dict(albums, orient="index")
        album_df.index.name = "album_name"

        all_tracks_df = pd.DataFrame()
        for album_id in tqdm(album_df['album_id'], desc=f"Processing {artist_name}"):
            time.sleep(1)
            tracks = get_tracks_for_album(token, album_id)
            if tracks is None:
                failed_requests = failed_requests.append({"Spotify ID": artist_id}, ignore_index=True)
                save_failed_requests(failed_requests)
                continue

            tracks = pd.DataFrame(tracks)
            tracks['album_id'] = album_id
            all_tracks_df = pd.concat([all_tracks_df, tracks], axis=0)

        af_df = get_audio_features(token, all_tracks_df['track_id'].tolist())
        af_df = af_df.rename(columns={'id': 'track_id'})
        all_aftracks_df = pd.merge(all_tracks_df, af_df, on='track_id')
        album_all_aftracks_df = pd.merge(album_df, all_aftracks_df, on='album_id')
        sanitized_artist_name = sanitize_filename(artist_name)
        album_all_aftracks_df.to_parquet(f'results/{sanitized_artist_name}_tracks.parquet')

        save_checkpoint(artist_id, artist_name)
        iteration_counter += 1
        if iteration_counter % sleep_interval == 0:
            time.sleep(10)
            
    except Exception as e:
        print(f"Error processing {artist_name}: {e}")
        failed_requests = failed_requests.append({"Spotify ID": artist_id}, ignore_index=True)
        save_failed_requests(failed_requests)
        continue

print("Completed processing all artists.")


Processing Popincourt:   0%|          | 0/2 [00:01<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
# max_retries = 3
# retry_counter = 0

# while not failed_requests.empty and retry_counter < max_retries:
#     print(f"Retrying failed requests (attempt {retry_counter + 1}/{max_retries})...")
#     retry_failed_requests = pd.DataFrame(columns=["Spotify ID"])
#     for _, row in failed_requests.iterrows():
#         artist_id = row["Spotify ID"]
#         artist_name = artist_ids[artist_id]
#         sanitized_artist_name = sanitize_filename(artist_name)
#         parquet_file_path = f'{sanitized_artist_name}_tracks.parquet'

#         if os.path.exists(parquet_file_path):
#             print(f"Parquet file already exists for {artist_name}, skipping...")
#             continue
#         try:
#             refreshed_token, new_token_start_time = refresh_token(token_start_time)
#             if refreshed_token:
#                 token = refreshed_token
#                 token_start_time = new_token_start_time
#             albums = get_albums_for_artist(token, artist_id)

#             if albums is None:
#                 failed_requests = failed_requests.append({"Spotify ID": artist_id}, ignore_index=True)
#                 save_failed_requests(failed_requests)
#                 continue

#             album_df = pd.DataFrame.from_dict(albums, orient="index")
#             album_df.index.name = "album_name"

#             all_tracks_df = pd.DataFrame()
#             for album_id in tqdm(album_df['album_id'], desc=f"Processing {artist_name}"):
#                 time.sleep(1)
#                 try:
#                     tracks = get_tracks_for_album(token, album_id)

#                     if tracks is None:
#                         failed_requests = failed_requests.append({"Spotify ID": artist_id}, ignore_index=True)
#                         save_failed_requests(failed_requests)
#                         continue

#                     tracks = pd.DataFrame(tracks)
#                     tracks['album_id'] = album_id
#                     all_tracks_df = pd.concat([all_tracks_df, tracks], axis=0)
#                 except Exception as e:
#                     print(f"Warning: Error fetching tracks for {artist_name}: {e}")

#             try:
#                 af_df = get_audio_features(token, all_tracks_df['track_id'].tolist())
#                 af_df = af_df.rename(columns={'id': 'track_id'})
#                 all_aftracks_df = pd.merge(all_tracks_df, af_df, on='track_id')
#                 album_all_aftracks_df = pd.merge(album_df, all_aftracks_df, on='album_id')
#                 sanitized_artist_name = sanitize_filename(artist_name)
#                 album_all_aftracks_df.to_parquet(f'{sanitized_artist_name}_tracks.parquet')

#                 save_checkpoint(artist_id, artist_name)
#             except Exception as e:
#                 print(f"Warning: Error fetching audio features for {artist_name}: {e}")

#             iteration_counter += 1
#             if iteration_counter % sleep_interval == 0:
#                 time.sleep(1)

#         except Exception as e:
#             print(f"Error processing {artist_name}: {e}")
#             retry_failed_requests = retry_failed_requests.append({"Spotify ID": artist_id}, ignore_index=True)

#     failed_requests = retry_failed_requests
#     retry_counter += 1

# if not failed_requests.empty:
#     print("Some requests still failed after all retries:")
#     print(failed_requests)
# else:
#     print("All failed requests have been successfully processed.")
